## Introduction to gurobipy
The purpose of this exercise is to introduce you to linear programming problems using the python API gurobipy. <br>
- Sections 1-3 are for you to familiarize yourselves with gurobi basics. 
- Section 4 introduces a more general way to formulate and solve optimization problems. 
- Section 5 gives a (very) basic introduction to object-oriented programming 
- Section 6 exemplifies how object-oriented programming can be used to structure optimization problems. <br>
<b>We recommend using this structure for the individual assignments and the group project.<b>

## 1 What is gurobipy?

- gurobipy is a python API used to formulate and solve mathematical optimization problems in python (similar to pyomo or JuMP in Julia).
- It is developed by the same company which develops the gurobi solver. 
- You can install gurobipy by running:

In [1]:
pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


- We import with prefix ```gp.```
- The specific module ```GRB``` is commonly imported separately, as it is used frequently. 

In [2]:
import gurobipy as gp
from gurobipy import GRB

## 2 Example: Simple problem

Let's use the following problem as an example:

$$
  \begin{align}
      \textrm{minimize} \quad &30x_1 + 20x_2 \\
      \textrm{subject to} \quad &0.6x_1 + 0.2x_2 \geq 60 \\
      &0.4x_1 + 0.8x_2 \geq 100 \\
      &x_1 \geq 0, x_2 \geq 0 \\
  \end{align}
$$

- We initialize a model object in which we'll store the problem.

In [3]:
model = gp.Model("My_LP_problem")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-07


- Now, we can add variables to the model with the method ```model.addVar(lb=0.0, ub=float('inf'), vtype=GRB.CONTINUOUS, name="")```.
- We can specify lower and upper bounds as well as domain using the arguments ```lb```, ```ub```, and ```vtype```, respectively.
- <b>Note that the default lower bound is 0!<b>

In [4]:
# Note that these two variables have the same bounds and domain
x_1 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x_1")
x_2 = model.addVar(name="x_2")

- Generally, we add constraints with the ```model.addConstr(constr, name="")``` method. 
- In this case, the constraints are linear and here, we should use the ```model.addLConstr(constr, name="")``` method.
- Here, it's important to store the constraints in a meaningful way so you can easily access specific dual variables after solving.
- Note, that in the ```GRB```module, you can find the three signs ```GRB.GREATER_EQUAL```, ```GRB.EQUAL```, and ```GRB.LESS_EQUAL```.

In [5]:
constraint_1 = model.addLConstr(0.6*x_1 + 0.2*x_2, GRB.GREATER_EQUAL, 60, name='constraint_1')
constraint_2 = model.addLConstr(0.4*x_1 + 0.8*x_2, GRB.GREATER_EQUAL, 100, name='constraint_2')

- We define the objective function with the method ```model.setObjective(expr, sense=None)```.
- <b>Remember to set the ```sense``` argument!<b>

In [6]:
model.setObjective(30*x_1 + 20*x_2, GRB.MINIMIZE)

- Now, we can solve the optimization problem with the method ```model.optimize```.

In [7]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: AMD Ryzen 7 7840U with Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x20d42a0c
Coefficient statistics:
  Matrix range     [2e-01, 8e-01]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 1e+02]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.600000e+02   0.000000e+00      0s
       2    3.9000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.900000000e+03


- We can check whether the problem was solved to optimality with ```model.status```.
- If so, we retrieve optimal objective function with ```model.ObjVal``` 
- and optimal primal and dual variable values with ```var.x``` and ```constr.Pi```, respectively.

In [8]:
if model.status == GRB.OPTIMAL:
    optimal_objective = model.ObjVal
    optimal_x_1 = x_1.x
    optimal_x_2 = x_2.x
    optimal_dual_1 = constraint_1.Pi
    optimal_dual_2 = constraint_2.Pi
    print(f"optimal objective: {optimal_objective}")
    print(f"optimal value of {x_1.VarName}: {optimal_x_1}")
    print(f"optimal value of {x_2.VarName}: {optimal_x_2}")
    print(f"optimal value of dual for {constraint_1.constrName}: {optimal_dual_1}")
    print(f"optimal value of dual for {constraint_2.constrName}: {optimal_dual_2}")
else:
    print(f"optimization of {model.ModelName} was not successful")

optimal objective: 3900.0
optimal value of x_1: 70.0
optimal value of x_2: 90.0
optimal value of dual for constraint_1: 40.0
optimal value of dual for constraint_2: 15.0


## 3 Task: Bestas' problem

Now, let's solve Bestas' problem from Exercise 1: 

$$
  \begin{align}
      \max_{x^E, x^H} \quad &10x^E + 15x^H \\
      \textrm{subject to} \quad &x^E + \frac{x^H}{0.8} \leq 200 \\
      &x^E \leq 100 \\
      &x^H \leq 100 \\
      &x^E \geq 0, x^H \geq 0
  \end{align}
$$

- In the same way as in step 2, please initialize and solve the problem using ```gurobipy```.

In [ ]:
# Initialize and solve problem here


## 4 Task: General formulation 

These problems can be expressed in a more general way by defining the inputs before-hand and making the rest of the code more general. <br>
Here is a general formulation of Morten's problem. 
- Please check that the solution corresponds to the one in section 2.

In [ ]:
# Set values of input parameters
VARIABLES = ['x1', 'x2']
objective_coeff = {'x1': 30, 'x2': 20} # Coefficients in objective function
constraints_coeff = {'x1': [0.6, 0.4], 'x2': [0.2, 0.8]} # Linear coefficients of constraints
constraints_rhs = [60, 100]# Right hand side coefficients of constraints
constraints_sense =  [GRB.GREATER_EQUAL, GRB.GREATER_EQUAL] # Direction of constraints

model = gp.Model("Mortens problem")

# Add variables to the Gurobi model
variables = {v: model.addVar(lb=0, name='Total production of CHP {0}'.format(v)) for v in VARIABLES}

# Set objective function and optimization direction of the Gurobi model
objective = gp.quicksum(objective_coeff[v] * variables[v] for v in VARIABLES)         
model.setObjective(objective, GRB.MINIMIZE)

# Add constraints to the Gurobi model
constraints = [
        (
                model.addLConstr(
                        gp.quicksum(constraints_coeff[v][i] * variables[v] for v in VARIABLES),
                        constraints_sense[i],
                        constraints_rhs[i]
                )
        ) for i in range(len(constraints_rhs))
]

# Optimize the Gurobi model
model.optimize()

# Check if the optimization was successful and print solutions
if model.status == GRB.OPTIMAL:
        print()
        print('-------------------   RESULTS  -------------------')
        optimal_variables = {v: variables[v].x for v in VARIABLES} # Save optimal values of decision variables
        optimal_objective = model.objVal # Save optimal value of objective function
        print("Optimal objective:", optimal_objective)
        for v in VARIABLES:
                print('Optimal {0}:'.format(v), optimal_variables[v])
else:
        print("Optimization was not successful")

- Now, use the general formulation to solve Bestas' problem (see section 3).

In [ ]:
# Use general formulation here



## 5 What is object-oriented programming?

- Object-oriented programming (OOP) is a very powerful tool to structure large optimization problems. 
- In this section, key concepts within OOP are introduced and in the next section, they are applied to the example problem from section 2.

#### 5.1 Classes 
OOP is all about classes. We'll use the class ```Dog``` (below) as a basis to discuss key concepts.

In [ ]:
class Dog:

    def __init__(self, breed: str, age: int):
        self.breed = breed
        self.age = age 
    
    def bark(self):
        if self.breed == 'Bloodhound':
            print("WOOF WOOF")
        elif self.breed == "Chihuahua":
            print("woof woof")
        else: 
            raise NotImplementedError("I don't know the bark of this dog")

#### 5.2 Instance
We can create an object which is an instance of the class by providing the arguments ```breed``` and ```age```.

In [ ]:
pluto = Dog('Bloodhound', 94)

#### 5.3 ```__init__``` method and attributes
When we created the instance ```pluto```, the ```self.__init__``` method was automatically called <br> 
and the two attributes ```self.breed``` and ```self.age``` were set. Here, ```self```refers to the instance. <br> 
We can access attributes outside of the class with ```instance.attribute```.

In [ ]:
print(pluto.breed)
print(pluto.age)

#### 5.4 methods
Functions defined inside the class are called methods and these can be performed on instances of the class.<br>
The methods often use (or alter) attributes like a dog's bark depends on its breed.

In [ ]:
print("Pluto barks: ")
pluto.bark()
harajuku = Dog("Chihuahua", 23)
print("Harajuku barks:")
harajuku.bark()

#### 5.5 Inheritance 

One class (let's call it class 1) can "extend" another class (class 2), which means it inherits <br>
the attributes and methods of class 2. Quite fittingly, class 2 is refered to as the parent class <br>
and class 1, the child class. The class definition looks like this: ```class Child(Parent):```. <br>
We continue the dog example below. 

In [ ]:
class Chihuahua(Dog):

    def __init__(self, age: int, shake: str):
        self.breed = "Chihuahua"
        self.age = age
        self.shake = shake

In [ ]:
tinkerbell = Chihuahua(14, 'strong')
tinkerbell.bark()

- Notice how we can use the method ```Dog.bark()``` as it is defined in the parent class ```Dog```,
- and how we introduced a new attribute ```shake``` which is specific to Chihuahuas. 

## 6 Example: Morten's problem with object-oriented programming

Admittedly, it is a bit over the top to use OOP for the example problem. However,<br> 
in the coming exercises and the project in particular, OOP will be a big help. 

- Firstly, we introduce a small class named ```Expando``` which allows for instance attributes to have attributes. (It will make sense later :))

In [ ]:
class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass

- Then, we define an ```InputData``` class which holds the necessary data for the optimization problem. 
- Therefore, it has attributes like ```self.VARIABLES```, ```self.objective_coeff```, ```self.constraints_coeff```, etc.

In [ ]:
class InputData:

    def __init__(
        self, 
        VARIABLES: list,
        objective_coeff: list[str, int],    # Coefficients in objective function
        constraints_coeff: list[str, int],  # Linear coefficients of constraints
        constraints_rhs: list[str, int],    # Right hand side coefficients of constraints
        constraints_sense: list[str, int],  # Direction of constraints
    ):
        self.VARIABLES = VARIABLES
        self.objective_coeff = objective_coeff
        self.constraints_coeff = constraints_coeff
        self.constraints_rhs = constraints_rhs
        self.constraints_sense = constraints_sense


- Now, we can define the class ```OptimizationProblem```, which takes an instance of the InputData class as the only argument and stores it as ```self.data```.
- It has methods to build and solve the problem as well as save and display results. 

In [ ]:
class OptimizationProblem():

    def __init__(self, input_data: InputData): # initialize class
        self.data = input_data # define data attributes
        self.results = Expando() # define results attributes
        self._build_model() # build gurobi model
    
    def _build_variables(self):
        self.variables = {v: self.model.addVar(lb=0, name='Total production of CHP {0}'.format(v)) for v in self.data.VARIABLES}
    
    def _build_constraints(self):
        self.constraints = [
            (
                self.model.addLConstr(
                        gp.quicksum(self.data.constraints_coeff[v][i] * self.variables[v] for v in self.data.VARIABLES),
                        self.data.constraints_sense[i],
                        self.data.constraints_rhs[i]
                )
            ) for i in range(len(self.data.constraints_rhs))
        ]

    def _build_objective_function(self):
        objective = gp.quicksum(self.data.objective_coeff[v] * self.variables[v] for v in self.data.VARIABLES)
        self.model.setObjective(objective, GRB.MINIMIZE)

    def _build_model(self):
        self.model = gp.Model(name='Economic dispatch')
        self._build_variables()
        self._build_objective_function()
        self._build_constraints()
        self.model.update()
    
    def _save_results(self):
        self.results.objective_value = self.model.ObjVal
        self.results.variables = {v: self.variables[v].x for v in self.data.VARIABLES}
        self.results.duals = [self.constraints[i].Pi for i in range(len(self.constraints))]

    def run(self):
        self.model.optimize()
        if self.model.status == GRB.OPTIMAL:
            self._save_results()
        else:
            print(f"optimization of {model.ModelName} was not successful")
    
    def display_results(self):
        print()
        print("-------------------   RESULTS  -------------------")
        print("Optimal objective value:")
        print(self.results.objective_value)
        print("Optimal variable values:")
        print(self.results.variables)
        print("Optimal dual values:")
        print(self.results.duals)

- Notice how ```self.results = Expando()``` allows us to save different results in the ```self.results``` attribute, e.g., ```self.results.objective_value```.

- Below is what corresponds to the ```main``` function where we create instances of the classes and use their methods. 

In [ ]:
# This corresponds to the main function
input_data = InputData(
    VARIABLES = ['x1', 'x2'],
    objective_coeff = {'x1': 30, 'x2': 20},
    constraints_coeff = {'x1': [0.6, 0.4], 'x2': [0.2, 0.8]},
    constraints_rhs = [60, 100],
    constraints_sense =  [GRB.GREATER_EQUAL, GRB.GREATER_EQUAL],
)
problem = OptimizationProblem(input_data)
problem.run()
problem.display_results()